In [1]:
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
import numpy as np
from sklearn.metrics import auc

# Function to calculate the Excess Mass (EM) curve and Area Under the Curve (AUC)
def em(t, t_max, volume_support, s_unif, s_X, n_generated):
    EM_t = np.zeros(t.shape[0])  # Initialize the EM_t array with zeros
    n_samples = s_X.shape[0]  # Get the number of samples in s_X
    s_X_unique = np.unique(s_X)  # Get the unique values from s_X
    EM_t[0] = 1.  # Set the first element of EM_t to 1

    for u in s_X_unique:
        # Update EM_t with the maximum value between the current EM_t and the new calculated value
        EM_t = np.maximum(
            EM_t,
            1. / n_samples * (s_X > u).sum() -
            t * (s_unif > u).sum() / n_generated * volume_support
        )

    # Find the index where EM_t is less than or equal to t_max
    amax = np.argmax(EM_t <= t_max) + 1

    # If amax is 1, it indicates t_max was not achieved
    if amax == 1:
        print('\n failed to achieve t_max \n')
        amax = -1

    # Calculate the AUC for the EM curve
    AUC = auc(t[:amax], EM_t[:amax])
    return AUC, EM_t, amax

# Function to calculate the Mass Volume (MV) curve and Area Under the Curve (AUC)
def mv(axis_alpha, volume_support, s_unif, s_X, n_generated):
    n_samples = s_X.shape[0]  # Get the number of samples in s_X
    s_X_argsort = np.argsort(s_X)  # Get the indices that would sort s_X
    mass = 0  # Initialize mass to zero
    cpt = 1  # Initialize counter to 1
    u = s_X[s_X_argsort[-1]]  # Get the maximum value from s_X
    mv = np.zeros(axis_alpha.shape[0])  # Initialize the mv array with zeros

    # Iterate through axis_alpha values
    for i in range(axis_alpha.shape[0]):
        while cpt <= n_samples:  # Ensure that cpt is within bounds
            u_index = -cpt
            if -cpt >= len(s_X_argsort):
                # Adjust index if out of bounds
                u_index = -len(s_X_argsort)

            u = s_X[s_X_argsort[u_index]]  # Get the value at the sorted index
            mass = 1. / n_samples * cpt  # Calculate mass

            if mass >= axis_alpha[i]:
                break  # Exit the loop if the required mass is reached

            cpt += 1

        # Calculate mv value
        mv[i] = float((s_unif >= u).sum()) / n_generated * volume_support

    # Calculate the AUC for the MV curve
    return auc(axis_alpha, mv), mv

# Function for extending our data of values from previous timestamps (temporalization)
def temporalize(X, y, lookback):
    output_X = []  # Initialize list for temporalized input data
    output_y = []  # Initialize list for temporalized output data

    # Iterate through the data, creating temporal sequences
    for i in range(len(X) - lookback - 1):
        t = []  # Initialize list for a single temporal sequence
        for j in range(1, lookback + 1):
            # Gather past records up to the lookback period
            t.append(X[i + j - 1])
        output_X.append(t)  # Append the sequence to the output_X list
        output_y.append(y[i + lookback + 1])  # Append the corresponding y value to output_y

    # Convert lists to numpy arrays and return them
    return np.array(output_X), np.array(output_y)


In [8]:
# !pip install pyod

name=""
folder="A2Benchmark"
file_path = 'data/'+folder+'/'

method='using yahoo synthetic data by temporalize+percentile+CNN'

In [4]:
# !pip install pyod


In [ ]:

evaluation_df = pd.DataFrame(all_evaluation_metrics)

# Save the DataFrame to a CSV file
evaluation_df.to_csv('csvfoldersynthetic/evaluation_metrics_'+method+'.csv', index=False)   